In [ ]:
pip install wordcloud

In [ ]:
import numpy as np
import pandas as pd
import re
import string
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from wordcloud import WordCloud,STOPWORDS,ImageColorGenerator
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import datetime
import warnings

In [ ]:
from google.colab import files
upload_file=files.upload()

Saving vaccination_all_tweets.csv to vaccination_all_tweets (1).csv


In [ ]:
df=pd.read_csv(r"vaccination_all_tweets.csv")

In [ ]:
df.head()

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
0,1.340540e+18,Rachel Roh,"La Crescenta-Montrose, CA",Aggregator of Asian American news; scanning di...,08-04-2009 17:52,405,1692,3247,False,20-12-2020 06:06,Same folks said daikon paste could treat a cyt...,['PfizerBioNTech'],Twitter for Android,0,0,False
1,1.338160e+18,Albert Fong,"San Francisco, CA","Marketing dude, tech geek, heavy metal & '80s ...",21-09-2009 15:27,834,666,178,False,13-12-2020 16:27,While the world has been on the wrong side of ...,NaN,Twitter Web App,1,1,False
2,1.337860e+18,eliðŸ‡±ðŸ‡¹ðŸ‡ªðŸ‡ºðŸ‘Œ,Your Bed,"heil, hydra ðŸ–â˜º",25-06-2020 23:30,10,88,155,False,12-12-2020 20:33,#coronavirus #SputnikV #AstraZeneca #PfizerBio...,"['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf...",Twitter for Android,0,0,False
3,1.337860e+18,Charles Adler,"Vancouver, BC - Canada","Hosting ""CharlesAdlerTonight"" Global News Radi...",10-09-2008 11:28,49165,3933,21853,True,12-12-2020 20:23,"Facts are immutable, Senator, even when you're...",NaN,Twitter Web App,446,2129,False
4,1.337850e+18,Citizen News Channel,NaN,Citizen News Channel bringing you an alternati...,23-04-2020 17:58,152,580,1473,False,12-12-2020 20:17,Explain to me again why we need a vaccine @Bor...,"['whereareallthesickpeople', 'PfizerBioNTech']",Twitter for iPhone,0,0,False


In [ ]:
df.shape

(228207, 16)

In [ ]:
df.describe()

,id,user_followers,user_friends,user_favourites,retweets,favorites
count,2.282070e+05,2.282070e+05,228207.000000,2.282070e+05,228207.000000,228207.000000
mean,1.407541e+18,9.605393e+04,986.736204,1.210417e+04,2.489284,10.819020
std,2.908485e+16,8.389053e+05,5293.500468,3.908233e+04,51.079821,199.527693
min,1.337730e+18,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000
25%,1.381940e+18,5.500000e+01,27.000000,5.700000e+01,0.000000,0.000000
50%,1.408130e+18,3.280000e+02,229.000000,9.960000e+02,0.000000,0.000000
75%,1.428690e+18,1.521000e+03,794.000000,7.300000e+03,1.000000,2.000000
max,1.463250e+18,1.635305e+07,582461.000000,1.299600e+06,12294.000000,54017.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228207 entries, 0 to 228206
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                228207 non-null  float64
 1   user_name         228205 non-null  object 
 2   user_location     161296 non-null  object 
 3   user_description  211189 non-null  object 
 4   user_created      228207 non-null  object 
 5   user_followers    228207 non-null  int64  
 6   user_friends      228207 non-null  int64  
 7   user_favourites   228207 non-null  int64  
 8   user_verified     228207 non-null  bool   
 9   date              228207 non-null  object 
 10  text              228207 non-null  object 
 11  hashtags          178504 non-null  object 
 12  source            228088 non-null  object 
 13  retweets          228207 non-null  int64  
 14  favorites         228207 non-null  int64  
 15  is_retweet        228207 non-null  bool   
dtypes: bool(2), float64(

In [ ]:
df.isnull().sum()

id                      0
user_name               2
user_location       66911
user_description    17018
user_created            0
user_followers          0
user_friends            0
user_favourites         0
user_verified           0
date                    0
text                    0
hashtags            49703
source                119
retweets                0
favorites               0
is_retweet              0
dtype: int64

In [ ]:
df['text']=df['text'].str.lower()
df['text']

0         same folks said daikon paste could treat a cyt...
1         while the world has been on the wrong side of ...
2         #coronavirus #sputnikv #astrazeneca #pfizerbio...
3         facts are immutable, senator, even when you're...
4         explain to me again why we need a vaccine @bor...
                                ...                        
228202    45+ #urban #bengaluru #covidvaccine availabili...
228203    18-44 #bbmp #bengaluru #covidvaccine availabil...
228204    18-44 #urban #bengaluru #covidvaccine availabi...
228205    they promote their vaccines leaving out the st...
228206    45+ #urban #bengaluru #covidvaccine availabili...
Name: text, Length: 228207, dtype: object

In [ ]:
import re

# Define a function to apply the regular expression substitution
def remove_urls(text):
    if isinstance(text, str):
        return re.sub(r"http\S+", "", text)
    else:
        return text

# Apply the function to the 'text' column
df['text'] = df['text'].apply(remove_urls)


In [ ]:
df['text']

0         same folks said daikon paste could treat a cyt...
1         while the world has been on the wrong side of ...
2         #coronavirus #sputnikv #astrazeneca #pfizerbio...
3         facts are immutable, senator, even when you're...
4         explain to me again why we need a vaccine @bor...
                                ...                        
228202    45+ #urban #bengaluru #covidvaccine availabili...
228203    18-44 #bbmp #bengaluru #covidvaccine availabil...
228204    18-44 #urban #bengaluru #covidvaccine availabi...
228205    they promote their vaccines leaving out the st...
228206    45+ #urban #bengaluru #covidvaccine availabili...
Name: text, Length: 228207, dtype: object

In [ ]:
import string

punctuation_removal = string.punctuation

def remove_punctuation(text):
    """Custom function to remove punctuation from a string."""
    if isinstance(text, str):
        return text.translate(str.maketrans('', '', punctuation_removal))
    else:
        return text

df["text"] = df["text"].apply(remove_punctuation)


In [ ]:
import re

def remove_non_alphabetic_chars(text):
    if isinstance(text, str):
        text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
        text = re.sub(r'\s+', ' ', text, flags=re.I)
        return text
    else:
        return text

df["text"] = df["text"].apply(remove_non_alphabetic_chars)


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

STOPWORDS = set(stopwords.words('english'))

def remove_stopwords(text):
    """Custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df["text"] = df["text"].apply(lambda text: remove_stopwords(text))
df["text"]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0         folks said daikon paste could treat cytokine s...
1         world wrong side history year hopefully bigges...
2         coronavirus sputnikv astrazeneca pfizerbiontec...
3         facts immutable senator even youre ethically s...
4         explain need vaccine borisjohnson matthancock ...
                                ...                        
228202    45 urban bengaluru covidvaccine availability 1...
228203    1844 bbmp bengaluru covidvaccine availability ...
228204    1844 urban bengaluru covidvaccine availability...
228205    promote vaccines leaving stronger russia vacci...
228206    45 urban bengaluru covidvaccine availability 1...
Name: text, Length: 228207, dtype: object

In [ ]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)
df["text"] = df["text"].apply(str)
df["text"] = df["text"].apply(remove_emoji)
df["text"]


0         folks said daikon paste could treat cytokine s...
1         world wrong side history year hopefully bigges...
2         coronavirus sputnikv astrazeneca pfizerbiontec...
3         facts immutable senator even youre ethically s...
4         explain need vaccine borisjohnson matthancock ...
                                ...                        
228202    45 urban bengaluru covidvaccine availability 1...
228203    1844 bbmp bengaluru covidvaccine availability ...
228204    1844 urban bengaluru covidvaccine availability...
228205    promote vaccines leaving stronger russia vacci...
228206    45 urban bengaluru covidvaccine availability 1...
Name: text, Length: 228207, dtype: object

In [ ]:
df["text"] = df["text"].apply(lambda x:re.sub(r'\s+[a-zA-Z]\s+', '', x))
df["text"] = df["text"].apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))
df["text"]

0         folks said daikon paste could treat cytokine s...
1         world wrong side history year hopefully bigges...
2         coronavirus sputnikv astrazeneca pfizerbiontec...
3         facts immutable senator even youre ethically s...
4         explain need vaccine borisjohnson matthancock ...
                                ...                        
228202    45 urban bengaluru covidvaccine availability 1...
228203    1844 bbmp bengaluru covidvaccine availability ...
228204    1844 urban bengaluru covidvaccine availability...
228205    promote vaccines leaving stronger russia vacci...
228206    45 urban bengaluru covidvaccine availability 1...
Name: text, Length: 228207, dtype: object

In [ ]:
from collections import Counter
cnt = Counter()
for text in df["text"].values:
    for word in text.split():
        cnt[word] += 1

cnt.most_common(10)

[('covaxin', 73937),
 ('vaccine', 53346),
 ('moderna', 47564),
 ('slots', 35480),
 ('dose', 31577),
 ('covid19', 30131),
 ('age', 25403),
 ('18', 21773),
 ('pfizer', 14495),
 ('bbmp', 14206)]

In [ ]:
import nltk
nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()

# Perform sentiment analysis on the 'text' column
df['sentiments'] = df["text"].apply(lambda x: sid.polarity_scores(' '.join(re.findall(r'\w+', str(x).lower()))))

# Extract the compound score and map it to sentiment categories
df['Sentiment'] = df['sentiments'].apply(lambda x: 'Fear' if x['compound'] < 0 else 'Joy' if x['compound'] > 0 else 'Neutral')

# Display the DataFrame with the new 'Sentiment' column
df.head()


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet,sentiments,Sentiment
0,1.340540e+18,Rachel Roh,"La Crescenta-Montrose, CA",Aggregator of Asian American news; scanning di...,08-04-2009 17:52,405,1692,3247,False,20-12-2020 06:06,folks said daikon paste could treat cytokine s...,['PfizerBioNTech'],Twitter for Android,0,0,False,"{'neg': 0.0, 'neu': 0.748, 'pos': 0.252, 'comp...",Joy
1,1.338160e+18,Albert Fong,"San Francisco, CA","Marketing dude, tech geek, heavy metal & '80s ...",21-09-2009 15:27,834,666,178,False,13-12-2020 16:27,world wrong side history year hopefully bigges...,NaN,Twitter Web App,1,1,False,"{'neg': 0.209, 'neu': 0.608, 'pos': 0.182, 'co...",Fear
2,1.337860e+18,eliðŸ‡±ðŸ‡¹ðŸ‡ªðŸ‡ºðŸ‘Œ,Your Bed,"heil, hydra ðŸ–â˜º",25-06-2020 23:30,10,88,155,False,12-12-2020 20:33,coronavirus sputnikv astrazeneca pfizerbiontec...,"['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf...",Twitter for Android,0,0,False,"{'neg': 0.0, 'neu': 0.846, 'pos': 0.154, 'comp...",Joy
3,1.337860e+18,Charles Adler,"Vancouver, BC - Canada","Hosting ""CharlesAdlerTonight"" Global News Radi...",10-09-2008 11:28,49165,3933,21853,True,12-12-2020 20:23,facts immutable senator even youre ethically s...,NaN,Twitter Web App,446,2129,False,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",Neutral
4,1.337850e+18,Citizen News Channel,NaN,Citizen News Channel bringing you an alternati...,23-04-2020 17:58,152,580,1473,False,12-12-2020 20:17,explain need vaccine borisjohnson matthancock ...,"['whereareallthesickpeople', 'PfizerBioNTech']",Twitter for iPhone,0,0,False,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",Neutral


In [ ]:
df['Number_Of_Words'] = df['text'].apply(lambda x:len(x.split(' ')))
#Average Word Length
df['Mean_Word_Length'] = df['text'].apply(lambda x:np.round(np.mean([len(w) for w in x.split(' ')]),2) )
df.head()

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet,sentiments,Sentiment,Number_Of_Words,Mean_Word_Length
0,1.340540e+18,Rachel Roh,"La Crescenta-Montrose, CA",Aggregator of Asian American news; scanning di...,08-04-2009 17:52,405,1692,3247,False,20-12-2020 06:06,folks said daikon paste could treat cytokine s...,['PfizerBioNTech'],Twitter for Android,0,0,False,"{'neg': 0.0, 'neu': 0.748, 'pos': 0.252, 'comp...",Joy,9,6.33
1,1.338160e+18,Albert Fong,"San Francisco, CA","Marketing dude, tech geek, heavy metal & '80s ...",21-09-2009 15:27,834,666,178,False,13-12-2020 16:27,world wrong side history year hopefully bigges...,NaN,Twitter Web App,1,1,False,"{'neg': 0.209, 'neu': 0.608, 'pos': 0.182, 'co...",Fear,11,6.09
2,1.337860e+18,eliðŸ‡±ðŸ‡¹ðŸ‡ªðŸ‡ºðŸ‘Œ,Your Bed,"heil, hydra ðŸ–â˜º",25-06-2020 23:30,10,88,155,False,12-12-2020 20:33,coronavirus sputnikv astrazeneca pfizerbiontec...,"['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf...",Twitter for Android,0,0,False,"{'neg': 0.0, 'neu': 0.846, 'pos': 0.154, 'comp...",Joy,12,7.75
3,1.337860e+18,Charles Adler,"Vancouver, BC - Canada","Hosting ""CharlesAdlerTonight"" Global News Radi...",10-09-2008 11:28,49165,3933,21853,True,12-12-2020 20:23,facts immutable senator even youre ethically s...,NaN,Twitter Web App,446,2129,False,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",Neutral,12,5.92
4,1.337850e+18,Citizen News Channel,NaN,Citizen News Channel bringing you an alternati...,23-04-2020 17:58,152,580,1473,False,12-12-2020 20:17,explain need vaccine borisjohnson matthancock ...,"['whereareallthesickpeople', 'PfizerBioNTech']",Twitter for iPhone,0,0,False,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",Neutral,7,11.71


In [ ]:
import nltk
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

def tokenization(text):
    text = re.split('\W+', text)
    return text

df['tokenized'] = df['text'].apply(lambda x: tokenization(x.lower()))

wn = WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

df['lemmatized'] = df['tokenized'].apply(lambda x: lemmatizer(x))
df.head()
df.tail()


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,...,source,retweets,favorites,is_retweet,sentiments,Sentiment,Number_Of_Words,Mean_Word_Length,tokenized,lemmatized
228202,1.460170e+18,VaxBLR,"Bengaluru, India",Hourly updates on FREE and PAID 18+ and 45+ va...,21-06-2021 08:44,31,0,0,False,15-11-2021 09:00,...,VaxBlr,0,0,False,"{'neg': 0.0, 'neu': 0.784, 'pos': 0.216, 'comp...",Joy,14,6.21,"[45, urban, bengaluru, covidvaccine, availabil...","[45, urban, bengaluru, covidvaccine, availabil..."
228203,1.460160e+18,VaxBLR,"Bengaluru, India",Hourly updates on FREE and PAID 18+ and 45+ va...,21-06-2021 08:44,31,0,0,False,15-11-2021 08:30,...,VaxBlr,0,1,False,"{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'comp...",Joy,12,7.25,"[1844, bbmp, bengaluru, covidvaccine, availabi...","[1844, bbmp, bengaluru, covidvaccine, availabi..."
228204,1.460160e+18,VaxBLR,"Bengaluru, India",Hourly updates on FREE and PAID 18+ and 45+ va...,21-06-2021 08:44,31,0,0,False,15-11-2021 08:30,...,VaxBlr,0,0,False,"{'neg': 0.0, 'neu': 0.784, 'pos': 0.216, 'comp...",Joy,14,6.29,"[1844, urban, bengaluru, covidvaccine, availab...","[1844, urban, bengaluru, covidvaccine, availab..."
228205,1.460160e+18,Gatti ValentinoðŸ¾,Southern Africa,"Entrepreneur, self taught cookðŸ²ðŸŒ® @Chelse...",28-08-2019 10:31,8103,3113,45726,False,15-11-2021 08:03,...,Twitter for Android,0,0,False,"{'neg': 0.0, 'neu': 0.49, 'pos': 0.51, 'compou...",Joy,7,7.29,"[promote, vaccines, leaving, stronger, russia,...","[promote, vaccine, leaving, stronger, russia, ..."
228206,1.460160e+18,VaxBLR,"Bengaluru, India",Hourly updates on FREE and PAID 18+ and 45+ va...,21-06-2021 08:44,31,0,0,False,15-11-2021 08:00,...,VaxBlr,0,0,False,"{'neg': 0.0, 'neu': 0.784, 'pos': 0.216, 'comp...",Joy,14,6.21,"[45, urban, bengaluru, covidvaccine, availabil...","[45, urban, bengaluru, covidvaccine, availabil..."


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['text'])
y = df['Sentiment']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, y_train)
rf_predictions = rf_classifier.predict(X_test)

# Evaluate Random Forest
accuracy = accuracy_score(y_test, rf_predictions)
precision = precision_score(y_test, rf_predictions, average='weighted')
recall = recall_score(y_test, rf_predictions, average='weighted')
f1 = f1_score(y_test, rf_predictions, average='weighted')

print("Random Forest Metrics:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Random Forest Metrics:
Accuracy: 0.9173787301169976
Precision: 0.9182876220129177
Recall: 0.9173787301169976
F1 Score: 0.9144393628280151


In [ ]:
dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train, y_train)
dt_predictions = dt_classifier.predict(X_test)

# Evaluate Decision Tree
accuracy = accuracy_score(y_test, dt_predictions)
precision = precision_score(y_test, dt_predictions, average='weighted')
recall = recall_score(y_test, dt_predictions, average='weighted')
f1 = f1_score(y_test, dt_predictions, average='weighted')

print("Decision Tree Metrics:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Decision Tree Metrics:
Accuracy: 0.8862451251040708
Precision: 0.8846031744317199
Recall: 0.8862451251040708
F1 Score: 0.8851282432628815


In [ ]:
df.shape

(228207, 22)

In [ ]:
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)
nb_predictions = nb_classifier.predict(X_test)

# Evaluate Naive Bayes
accuracy = accuracy_score(y_test, nb_predictions)
precision = precision_score(y_test, nb_predictions, average='weighted')
recall = recall_score(y_test, nb_predictions, average='weighted')
f1 = f1_score(y_test, nb_predictions, average='weighted')

print("Naive Bayes Metrics:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Naive Bayes Metrics:
Accuracy: 0.824350379036852
Precision: 0.8310594855064897
Recall: 0.824350379036852
F1 Score: 0.8231279314250824


In [ ]:
gb_classifier = GradientBoostingClassifier()
gb_classifier.fit(X_train, y_train)
gb_predictions = gb_classifier.predict(X_test)

# Evaluate Gradient Boosting
accuracy = accuracy_score(y_test, gb_predictions)
precision = precision_score(y_test, gb_predictions, average='weighted')
recall = recall_score(y_test, gb_predictions, average='weighted')
f1 = f1_score(y_test, gb_predictions, average='weighted')

print("Gradient Boosting Metrics:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Gradient Boosting Metrics:
Accuracy: 0.7683931466631612
Precision: 0.8189883354635852
Recall: 0.7683931466631612
F1 Score: 0.7489980127191147


In [ ]:
nn_classifier = MLPClassifier()
nn_classifier.fit(X_train, y_train)
nn_predictions = nn_classifier.predict(X_test)

accuracy = accuracy_score(y_test, nn_predictions)
precision = precision_score(y_test, nn_predictions, average='weighted')
recall = recall_score(y_test, nn_predictions, average='weighted')
f1 = f1_score(y_test, nn_predictions, average='weighted')

print("Neural Networks Metrics:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Neural Networks Metrics:
Accuracy: 0.9242145392401735
Precision: 0.9234058789666741
Recall: 0.9242145392401735
F1 Score: 0.9231256532124533
